# Redes Neurais Auto Encoders

### Descriçao

---
1. O objetivo deste notebook é construir uma Rede Neural Auto Enconders.
2. Utilizaremos o conjunto de dados direto do github, a qual iremos baixar e usar.

3. O problema consiste em prever agrupar e criar sistemas de recomendações a partir de avaliações de filmes
---

# Instalação dos pacotes

In [1]:
!pip install pandas numpy scikit-learn keras torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Obtendo o dataset

In [2]:
# Clona o repositório do GitHub que contém o material necessário
!git clone https://github.com/batestin1/coding_the_future_dio_redes_neurais.git

# Move apenas a pasta 'dataset' para o diretório atual (/content/)
!mv coding_the_future_dio_redes_neurais/dataset /content/

# Remove o restante do repositório clonado, já que não é necessário
!rm -rf coding_the_future_dio_redes_neurais

Cloning into 'coding_the_future_dio_redes_neurais'...
remote: Enumerating objects: 11037, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 11037 (delta 50), reused 74 (delta 24), pack-reused 10932 (from 1)
Receiving objects: 100% (11037/11037), 196.13 MiB | 31.10 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Updating files: 100% (11028/11028), done.


# Instalando as bibliotecas




In [3]:
# Importa a biblioteca NumPy para operações numéricas e manipulação de arrays
import numpy as np

# Importa a biblioteca pandas para análise e manipulação de dados
import pandas as pd

# Importa a biblioteca os para interações com o sistema operacional (como manipulação de arquivos e diretórios)
import os

# Importa a biblioteca principal do PyTorch para operações de tensores
import torch

# Importa o submódulo do PyTorch para construção de redes neurais
import torch.nn as nn

# Importa o submódulo do PyTorch para treinamento paralelo de redes neurais
import torch.nn.parallel

# Importa o submódulo do PyTorch que contém algoritmos de otimização
import torch.optim as optim

# Importa o submódulo do PyTorch para manipulação de conjuntos de dados
import torch.utils.data

# Importa a classe Variable do PyTorch para diferenciação automática (autograd)
from torch.autograd import Variable

# Importa a função do Keras para carregar modelos pré-treinados
from keras.models import load_model

# Importa a função do TensorFlow Keras para salvar modelos
from tensorflow.keras.models import load_model

# Lendo os Dataset

In [4]:
# Importa o arquivo 'movies.dat' contendo informações sobre os filmes
movies = pd.read_csv(
  '/content/dataset/ae/movies.dat',  # Caminho do arquivo
  sep='::',  # Define o separador como '::'
  header=None,  # Indica que o arquivo não possui cabeçalho
  engine='python',  # Usa o engine Python para suportar separadores incomuns
  encoding='latin-1'  # Define o encoding para leitura correta dos caracteres
)

# Importa o arquivo 'users.dat' contendo informações sobre os usuários
users = pd.read_csv(
  '/content/dataset/ae/users.dat',
  sep='::',
  header=None,
  engine='python',
  encoding='latin-1'
)

# Importa o arquivo 'ratings.dat' contendo informações sobre as avaliações dos usuários
ratings = pd.read_csv(
  '/content/dataset/ae/ratings.dat',
  sep='::',
  header=None,
  engine='python',
  encoding='latin-1'
)

In [5]:
movies.head(3)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [6]:
movies.columns = ['indice_filme', 'nome_filme', 'genero_filme']
movies.head(3)

,indice_filme,nome_filme,genero_filme
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [7]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
users.columns = ['indice_user', 'genero_user', 'idade_user', 'cod_profissao', 'cep']
users.head()

,indice_user,genero_user,idade_user,cod_profissao,cep
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
ratings.columns = ['ind_user', 'indice_filme', 'avaliacao', 'data']
ratings.head()

,ind_user,indice_filme,avaliacao,data
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
# Cria uma cópia do DataFrame 'ratings' para visualização e manipulação
visualizando_data = ratings.copy()

# Converte a coluna 'data' (timestamp) de segundos para o formato de data e hora
visualizando_data['data'] = pd.to_datetime(
  visualizando_data['data'],  # Coluna de timestamps
  unit='s'  # Unidade em segundos
)

# Exibe as primeiras 5 linhas do DataFrame para inspeção
visualizando_data.head()

,ind_user,indice_filme,avaliacao,data
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


# Preparando os dados de treino e os dados de teste

In [12]:
# Carrega o conjunto de dados de treinamento a partir de um arquivo CSV
training_set = pd.read_csv('/content/dataset/ae/train.csv')

# Converte o conjunto de dados de treinamento em um array NumPy com o tipo de dados 'int'
training_set = np.array(training_set, dtype='int')

# Carrega o conjunto de dados de teste a partir de um arquivo CSV
test_set = pd.read_csv('/content/dataset/ae/test.csv')

# Converte o conjunto de dados de teste em um array NumPy com o tipo de dados 'int'
test_set = np.array(test_set, dtype='int')

In [13]:
training_set.shape

(750121, 4)

In [14]:
training_set[0]

array([        1,       661,         3, 978302109])

In [15]:
test_set.shape

(250088, 4)

In [16]:
test_set[0]

array([        1,      1193,         5, 978300760])

In [17]:
training_set[:,0].max()

np.int64(6040)

In [18]:
test_set[:,1].max()

np.int64(3952)

In [19]:
# Quantidade de usuários e filmes

# Encontra o maior ID de usuário no conjunto de treinamento (assumido como o número total de usuários)
nb_users = int(training_set[:, 0].max())

# Encontra o maior ID de filme no conjunto de treinamento (assumido como o número total de filmes)
nb_movies = int(training_set[:, 1].max())

In [20]:
nb_users

6040

In [21]:
nb_movies

3952

In [22]:
# Função para converter os dados em uma matriz com usuários nas linhas e filmes nas colunas
# (iremos criar uma lista onde cada linha representa um usuário e cada coluna um filme)

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):  # Itera sobre cada ID de usuário
    id_movies = data[:, 1][data[:, 0] == id_users]  # Obtém os IDs dos filmes avaliados pelo usuário atual
    id_ratings = data[:, 2][data[:, 0] == id_users]  # Obtém as avaliações correspondentes feitas pelo usuário atual
    ratings = np.zeros(nb_movies)  # Cria um vetor de zeros com o número total de filmes
    ratings[id_movies - 1] = id_ratings  # Atribui as avaliações nos índices correspondentes (ajustando o índice)
    new_data.append(list(ratings))  # Adiciona o vetor de avaliações à nova lista
  return new_data  # Retorna a matriz com as avaliações

# Converte o conjunto de dados de treinamento usando a função definida
training_set = convert(training_set)

# Converte o conjunto de dados de teste usando a função definida
test_set = convert(test_set)

In [23]:
type(training_set)

list

In [24]:
len(training_set)

6040

In [25]:
len(test_set)

6040

#### Essa padronizacao do mesmo tamanho do arquivo de treino e teste significa que apesar do usuario ter ou não avaliado o filme, ele vai estar no arquivo e nossa rede vai aprender com isso. caso nao tenha avaliado, preenchemos com 0

# Criando Tensors do Torch


##### Aqui vamos criar os tensores do toach. Matrizes multidimensionais de um unico tipo, em nosso caso, float

In [26]:
# Converte o conjunto de dados de treinamento para um tensor de ponto flutuante do PyTorch
training_set = torch.FloatTensor(training_set)

# Converte o conjunto de dados de teste para um tensor de ponto flutuante do PyTorch
test_set = torch.FloatTensor(test_set)

In [27]:
training_set

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.]])

# Criando a arquitetura da AE

In [28]:
# Definição da classe da rede de Autoencoder
class redes_auto_enconder(nn.Module):  # Utiliza herança da classe nn.Module do PyTorch
  def __init__(self):
    super(redes_auto_enconder, self).__init__()  # Inicializa a superclasse nn.Module

    # Define a primeira camada totalmente conectada:
    # Entrada: número total de filmes (nb_movies), Saída: 20 neurônios
    self.fc1 = nn.Linear(nb_movies, 20)

    # Segunda camada totalmente conectada:
    # Entrada: 20 neurônios, Saída: 10 neurônios
    self.fc2 = nn.Linear(20, 10)

    # Terceira camada totalmente conectada:
    # Entrada: 10 neurônios, Saída: 20 neurônios
    self.fc3 = nn.Linear(10, 20)

    # Quarta camada totalmente conectada:
    # Entrada: 20 neurônios, Saída: número total de filmes (nb_movies)
    self.fc4 = nn.Linear(20, nb_movies)

    # Função de ativação Sigmoid para aplicar não-linearidade
    self.activation = nn.Sigmoid()

  def forward(self, x):
    # Define a passagem para frente (forward pass) da rede
    x = self.activation(self.fc1(x))  # Primeira camada + ativação
    x = self.activation(self.fc2(x))  # Segunda camada + ativação
    x = self.activation(self.fc3(x))  # Terceira camada + ativação
    x = self.fc4(x)  # Quarta camada (sem ativação final)
    return x  # Retorna a saída final


# Instancia a rede autoencoder
rae = redes_auto_enconder()

# Define a função de perda (loss function) como erro quadrático médio (MSE)
criterion = nn.MSELoss()

# Define o otimizador RMSprop para atualização dos pesos
optimizer = optim.RMSprop(
  rae.parameters(),  # Passa os parâmetros da rede para o otimizador
  lr=0.01,            # Taxa de aprendizado
  weight_decay=0.5    # Fator de decaimento de peso para regularização
)

# Treinando a AE

In [29]:
# Define o número de épocas (iterações completas sobre o conjunto de dados)
nb_epoch = 8  # (pode usar 800 para melhor performance)

# Loop principal de treinamento
for epoch in range(1, nb_epoch + 1):

  train_loss = 0  # Variável para acumular a perda (loss) da época
  s = 0.0         # Contador de usuários que avaliaram pelo menos um filme

  # Itera sobre todos os usuários
  for id_user in range(nb_users):

    # Cria o vetor de entrada para o usuário atual e adiciona uma dimensão extra
    input = Variable(training_set[id_user]).unsqueeze(0)

    # Clona o vetor de entrada para ser usado como alvo (target)
    target = input.clone()

    # Só treina usuários que avaliaram pelo menos um filme
    if torch.sum(target.data > 0) > 0:

      # Passa o input pela rede para obter o output
      output = rae(input)

      # Desabilita o cálculo de gradientes para o target
      target.require_grad = False

      # Garante que as saídas dos filmes não avaliados sejam zeradas
      output[target == 0] = 0

      # Calcula a perda entre a saída e o valor real
      loss = criterion(output, target)

      # Fator de correção para normalizar a perda pelo número de filmes avaliados
      mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)

      # Retropropaga o erro
      loss.backward()

      # Acumula a perda corrigida
      train_loss += np.sqrt(loss.data * mean_corrector)

      # Incrementa o número de usuários considerados
      s += 1.

      # Atualiza os pesos da rede
      optimizer.step()

  # Imprime o progresso da época atual
  print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

<ipython-input-29-7579fede92de>:41: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  train_loss += np.sqrt(loss.data * mean_corrector)


epoch: 1 loss: tensor(1.3478)
epoch: 2 loss: tensor(1.0101)
epoch: 3 loss: tensor(0.9900)
epoch: 4 loss: tensor(0.9832)
epoch: 5 loss: tensor(0.9802)
epoch: 6 loss: tensor(0.9783)
epoch: 7 loss: tensor(0.9772)
epoch: 8 loss: tensor(0.9765)


# Testando a Rede

In [30]:
# Inicializa a variável de perda de teste
test_loss = 0
# Inicializa a contagem de usuários com dados válidos
s = 0.

# Loop sobre todos os usuários
for id_user in range(nb_users):

  # Cria o vetor de entrada do usuário a partir do conjunto de treino
  input = Variable(training_set[id_user]).unsqueeze(0)

  # Cria o vetor de alvo (target) do usuário a partir do conjunto de teste
  target = Variable(test_set[id_user]).unsqueeze(0)

  # Só testa usuários que avaliaram pelo menos um filme no conjunto de teste
  if torch.sum(target.data > 0) > 0:

    # Passa o input pela rede para obter a saída
    output = rae(input)

    # Desabilita o cálculo de gradientes para o target
    target.require_grad = False

    # Garante que as saídas correspondentes a filmes não avaliados sejam zeradas
    output[target == 0] = 0

    # Calcula a perda entre a saída e o valor real
    loss = criterion(output, target)

    # Fator de correção para normalizar a perda pelo número de filmes avaliados
    mean_corrector = nb_movies / float(torch.sum(target.data > 0))

    # Retropropaga o erro (não é obrigatório no teste, mas não atrapalha neste caso)
    loss.backward()

    # Acumula a perda corrigida
    test_loss += np.sqrt(loss.data * mean_corrector)

    # Incrementa o número de usuários considerados
    s += 1.

# Imprime a perda média no conjunto de teste
print('Loss de teste: ' + str(test_loss / s))

<ipython-input-30-6698b7380e2e>:37: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  test_loss += np.sqrt(loss.data * mean_corrector)


Loss de teste: tensor(0.9725)


# Salvando o modelo

In [31]:
folder = 'ae/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
  os.makedirs(folder)

# Salva o modelo treinado no diretório especificado
torch.save(rae.state_dict(), os.path.join(folder, 'modelo_autoencoder.pth'))

# Importando os modelos

In [32]:
# Carrega o estado do modelo a partir do arquivo salvo no diretório especificado

# Antes de carregar, você precisa instanciar uma nova rede
model = redes_auto_enconder()
model.eval()  # Coloca o modelo em modo de avaliação (importante para desativar dropout, batchnorm, etc.)

# Carrega os pesos salvos do modelo
model.load_state_dict(torch.load(os.path.join(folder, 'modelo_autoencoder.pth')))

<All keys matched successfully>

In [33]:
model

redes_auto_enconder(
  (fc1): Linear(in_features=3952, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=3952, bias=True)
  (activation): Sigmoid()
)

In [34]:
test_loss = 0  # Inicializa a variável de perda de teste
s = 0.0  # Inicializa o contador de usuários válidos

for id_user in range(nb_users):  # Loop sobre cada usuário
  input = Variable(test_set[id_user]).unsqueeze(0)  # Prepara a entrada (dados de teste do usuário atual) adicionando uma dimensão
  target = Variable(test_set[id_user]).unsqueeze(0)  # Prepara o alvo (dados reais de teste do usuário atual)

  if torch.sum(target.data > 0) > 0:  # Verifica se o usuário avaliou pelo menos um filme
    output = model(input)  # Usa o modelo carregado para gerar a saída (predição)
    target.require_grad = False  # Desativa o cálculo de gradiente para o target, pois não vamos treinar agora
    output[target == 0] = 0  # Define a saída como 0 onde o target também for 0 (filmes não avaliados)
    loss = criterion(output, target)  # Calcula a perda (erro) entre a saída e o valor real
    mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)  # Fator de correção para normalizar o erro
    test_loss += np.sqrt(loss.data * mean_corrector)  # Acumula a perda corrigida
    s += 1.0  # Incrementa o contador de usuários válidos

# Imprime a perda média de teste para todos os usuários
print('Test loss: ' + str(test_loss / s))

<ipython-input-34-e85a4a678337>:14: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  test_loss += np.sqrt(loss.data * mean_corrector)  # Acumula a perda corrigida


Test loss: tensor(0.9726)
